<a href="https://colab.research.google.com/github/Franciscotor1/Fraud-detection-Spark/blob/master/frauddetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Descarga e instala Apache Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

# Configura las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

# Añade Spark al sistema
import findspark
findspark.init()

# Crea una sesión Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("example").getOrCreate()


In [ ]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("FraudDetection").getOrCreate()

# Cargar el conjunto de datos desde el archivo CSV en Colab
file_path = "Fraud-content.csv"  # Asegúrate de que el archivo esté en el mismo directorio que tu notebook Colab
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Mostrar el esquema y las primeras filas del DataFrame
df.printSchema()
df.show(5)


root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PA

Paso 1:
Transacciones fraudulentas en el Csv

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crear una sesión de Spark
spark = SparkSession.builder.appName("FraudDetection").getOrCreate()

# Cargar el conjunto de datos desde el archivo CSV en Colab
file_path = "Fraud-content.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Filtrar las transacciones fraudulentas
fraudulent_transactions = df.filter(col("isFraud") == 1)

# Mostrar todas las transacciones fraudulentas
fraudulent_transactions.show(truncate=False)


+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|type    |amount    |nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|1   |TRANSFER|181.0     |C1305486145|181.0        |0.0           |C553264065 |0.0           |0.0           |1      |0             |
|1   |CASH_OUT|181.0     |C840083671 |181.0        |0.0           |C38997010  |21182.0       |0.0           |1      |0             |
|1   |TRANSFER|2806.0    |C1420196421|2806.0       |0.0           |C972765878 |0.0           |0.0           |1      |0             |
|1   |CASH_OUT|2806.0    |C2101527076|2806.0       |0.0           |C1007251739|26202.0       |0.0           |1      |0             |
|1   |TRANSFER|20128.0   |C137533655 |20128.0      |0.0           |C1

In [ ]:
# A) Montos más Altos de Fraudes
highest_fraud_amounts = fraudulent_transactions.orderBy(col("amount").desc())
highest_fraud_amounts.show(10, truncate=False)


+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|type    |amount    |nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|4   |TRANSFER|1.0E7     |C7162498   |1.293041844E7|2930418.44    |C945327594 |0.0           |0.0           |1      |0             |
|19  |CASH_OUT|1.0E7     |C2050703310|1.0E7        |0.0           |C1622860679|504326.62     |1.03424179E7  |1      |0             |
|19  |TRANSFER|1.0E7     |C416779475 |1.186100832E7|1861008.32    |C380259496 |0.0           |0.0           |1      |0             |
|33  |TRANSFER|1.0E7     |C1439740840|1.988781906E7|9887819.06    |C875288652 |0.0           |0.0           |1      |0             |
|33  |CASH_OUT|1.0E7     |C29118015  |1.0E7        |0.0           |C1

In [ ]:
# Filtrar las transacciones fraudulentas
fraudulent_transactions = df.filter(col("isFraud") == 1)

# B) Monto Menos Elevado de Fraudes
lowest_fraud_amounts = fraudulent_transactions.orderBy(col("amount"))
lowest_fraud_amounts.show(10, truncate=False)


+----+--------+------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|type    |amount|nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|15  |CASH_OUT|119.0 |C1584512618|119.0        |0.0           |C2102058838|4844060.13    |4844179.13    |1      |0             |
|15  |TRANSFER|119.0 |C1995557473|119.0        |0.0           |C1480876722|0.0           |0.0           |1      |0             |
|7   |TRANSFER|164.0 |C1455969984|164.0        |0.0           |C2119910556|0.0           |0.0           |1      |0             |
|7   |CASH_OUT|164.0 |C1173659886|164.0        |0.0           |C1769947269|4068.0        |0.0           |1      |0             |
|15  |TRANSFER|170.0 |C553034695 |170.0        |0.0           |C26381896  |0.0           |0.0    

In [ ]:
# A) Montos más Altos de Fraudes
highest_fraud_amounts = fraudulent_transactions.orderBy(col("amount").desc())
highest_fraud_amounts.show(10, truncate=False)


+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|type    |amount    |nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|4   |TRANSFER|1.0E7     |C7162498   |1.293041844E7|2930418.44    |C945327594 |0.0           |0.0           |1      |0             |
|19  |CASH_OUT|1.0E7     |C2050703310|1.0E7        |0.0           |C1622860679|504326.62     |1.03424179E7  |1      |0             |
|19  |TRANSFER|1.0E7     |C416779475 |1.186100832E7|1861008.32    |C380259496 |0.0           |0.0           |1      |0             |
|33  |TRANSFER|1.0E7     |C1439740840|1.988781906E7|9887819.06    |C875288652 |0.0           |0.0           |1      |0             |
|33  |CASH_OUT|1.0E7     |C29118015  |1.0E7        |0.0           |C1

In [ ]:
output_path = "/content/higherfrauds.csv"

# A) Montos más Altos de Fraudes
highest_fraud_amounts = fraudulent_transactions.orderBy(col("amount").desc())
highest_fraud_amounts.show(10, truncate=False)

# Guardar el DataFrame en un archivo CSV
highest_fraud_amounts.write.csv(output_path, header=True, mode="overwrite")

print(f"El DataFrame se ha guardado exitosamente en: {output_path}")


+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|type    |amount    |nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|4   |TRANSFER|1.0E7     |C7162498   |1.293041844E7|2930418.44    |C945327594 |0.0           |0.0           |1      |0             |
|19  |CASH_OUT|1.0E7     |C2050703310|1.0E7        |0.0           |C1622860679|504326.62     |1.03424179E7  |1      |0             |
|19  |TRANSFER|1.0E7     |C416779475 |1.186100832E7|1861008.32    |C380259496 |0.0           |0.0           |1      |0             |
|33  |TRANSFER|1.0E7     |C1439740840|1.988781906E7|9887819.06    |C875288652 |0.0           |0.0           |1      |0             |
|33  |CASH_OUT|1.0E7     |C29118015  |1.0E7        |0.0           |C1

In [ ]:
output_path_single_file = "/content/higherfrauds_single_file.csv"

# A) Montos más Altos de Fraudes
highest_fraud_amounts = fraudulent_transactions.orderBy(col("amount").desc())
highest_fraud_amounts.show(10, truncate=False)

# Guardar el DataFrame en un solo archivo CSV
highest_fraud_amounts.coalesce(1).write.csv(output_path_single_file, header=True, mode="overwrite")

print(f"El DataFrame se ha guardado exitosamente en: {output_path_single_file}")


+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|type    |amount    |nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|4   |TRANSFER|1.0E7     |C7162498   |1.293041844E7|2930418.44    |C945327594 |0.0           |0.0           |1      |0             |
|19  |CASH_OUT|1.0E7     |C2050703310|1.0E7        |0.0           |C1622860679|504326.62     |1.03424179E7  |1      |0             |
|19  |TRANSFER|1.0E7     |C416779475 |1.186100832E7|1861008.32    |C380259496 |0.0           |0.0           |1      |0             |
|33  |TRANSFER|1.0E7     |C1439740840|1.988781906E7|9887819.06    |C875288652 |0.0           |0.0           |1      |0             |
|33  |CASH_OUT|1.0E7     |C29118015  |1.0E7        |0.0           |C1

In [ ]:
highest_fraud_amounts.show()


+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|    amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|  33|TRANSFER|     1.0E7|C1439740840|1.988781906E7|    9887819.06| C875288652|           0.0|           0.0|      1|             0|
|  33|CASH_OUT|     1.0E7|  C29118015|        1.0E7|           0.0|C1379703840|           0.0|         1.0E7|      1|             0|
|  19|TRANSFER|     1.0E7| C416779475|1.186100832E7|    1861008.32| C380259496|           0.0|           0.0|      1|             0|
|   4|TRANSFER|     1.0E7|   C7162498|1.293041844E7|    2930418.44| C945327594|           0.0|           0.0|      1|             0|
|  19|CASH_OUT|     1.0E7|C2050703310|        1.0E7|           0.0|C1

In [ ]:
from pyspark.sql.functions import col

# A) Montos más Altos de Fraudes
highest_fraud_amounts = fraudulent_transactions.orderBy(col("amount").desc())

# Convertir la columna "amount" a tipo Decimal
highest_fraud_amounts = highest_fraud_amounts.withColumn("amount", col("amount").cast("decimal(18, 2)"))

# Mostrar el DataFrame con los montos convertidos
highest_fraud_amounts.show(10, truncate=False)


+----+--------+-----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|type    |amount     |nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+-----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|4   |TRANSFER|10000000.00|C7162498   |1.293041844E7|2930418.44    |C945327594 |0.0           |0.0           |1      |0             |
|19  |CASH_OUT|10000000.00|C2050703310|1.0E7        |0.0           |C1622860679|504326.62     |1.03424179E7  |1      |0             |
|19  |TRANSFER|10000000.00|C416779475 |1.186100832E7|1861008.32    |C380259496 |0.0           |0.0           |1      |0             |
|33  |TRANSFER|10000000.00|C1439740840|1.988781906E7|9887819.06    |C875288652 |0.0           |0.0           |1      |0             |
|33  |CASH_OUT|10000000.00|C29118015  |1.0E7        |0.0      

In [ ]:
from pyspark.sql.functions import col

# Asumiendo que tu DataFrame se llama highest_fraud_amounts
highest_fraud_amounts = highest_fraud_amounts.withColumn("amount", col("amount").cast("double"))

# Mostrar el DataFrame con los montos convertidos
highest_fraud_amounts.show(10, truncate=False)


+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|type    |amount    |nameOrig   |oldbalanceOrg|newbalanceOrig|nameDest   |oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+----------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|4   |TRANSFER|1.0E7     |C7162498   |1.293041844E7|2930418.44    |C945327594 |0.0           |0.0           |1      |0             |
|19  |CASH_OUT|1.0E7     |C2050703310|1.0E7        |0.0           |C1622860679|504326.62     |1.03424179E7  |1      |0             |
|19  |TRANSFER|1.0E7     |C416779475 |1.186100832E7|1861008.32    |C380259496 |0.0           |0.0           |1      |0             |
|33  |TRANSFER|1.0E7     |C1439740840|1.988781906E7|9887819.06    |C875288652 |0.0           |0.0           |1      |0             |
|33  |CASH_OUT|1.0E7     |C29118015  |1.0E7        |0.0           |C1

In [ ]:
import pandas as pd

# Convertir el DataFrame de Spark a Pandas
pandas_df = highest_fraud_amounts.toPandas()

# Guardar el DataFrame de Pandas en un archivo CSV
pandas_df.to_csv("/content/topandas.csv", index=False)


In [ ]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("FraudDetection").getOrCreate()

# Cargar el conjunto de datos desde el archivo CSV en Colab
file_path = "Fraud-content.csv"  # Asegúrate de que el archivo esté en el mismo directorio que tu notebook Colab
new_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Mostrar el esquema y las primeras filas del DataFrame
df.printSchema()
df.show(5)

ModuleNotFoundError: No module named 'pyspark'